# Importing in ilcd format

In [1]:
from collections import Counter
from pathlib import Path
import bw2io
import bw2calc
import bw2data
from bw2io.importers.ilcd import ILCDImporter
import pandas as pd


import importlib


In [2]:
print(bw2io.__version__)

(0, 9, 'DEV10')


In [3]:
gwp100 = ('IPCC 2013', 'climate change', 'GWP 100a')

In [4]:
bw2data.projects.set_current('ilcd_import')
bw2io.bw2setup()

Biosphere database already present!!! No setup is needed


In [5]:
bw2data.databases

Databases dictionary with 9 object(s):
	biosphere3
	example_glad
	example_glad_many
	example_ilcd
	example_ilcd_lcacommons
	lca_commons
	lcacommons_electricity
	lcacommons_forest
	test_db

In [6]:
b3 = bw2data.Database('biosphere3')

In [5]:
path_to_examples = Path('bw2io/data/examples/ilcd_examples')
assert path_to_examples.is_dir()
paths_to_examples = list(path_to_examples.glob('*zip'))

## 1) GLAD - single activity

this activity is used for testing purposes. The pdfs have been removed from the zip file to make it lighter

In [6]:
path_to_GLAD_example = Path('bw2io/data/examples/ilcd_example.zip')

In [7]:
so = ILCDImporter(dirpath= path_to_GLAD_example, dbname='example_glad')

In [8]:
so.apply_strategies()
so.match_database('biosphere3',fields=['database','code'])
so.match_database(fields=['database','code'])
so.statistics()

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applying strategy: set_activity_parameters
Applied 8 strategies in 0.03 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
15 exchanges
4 unlinked exchanges
  Type biosphere: 2 unique unlinked exchanges
  Type technosphere: 2 unique unlinked exchanges


(1, 15, 4)

In [9]:
unlinked_df = pd.DataFrame(list(so.unlinked))

In [10]:
unlinked_df.loc[unlinked_df['type']=='biosphere',:].T

,0,3
uuid,08a91e70-3ddc-11dd-960b-0050c2490048,fe0acd60-3ddc-11dd-af54-0050c2490048
category_0,Emissions,Emissions
category_1,Emissions to air,Emissions to air
category_2,"Emissions to air, unspecified","Emissions to air, unspecified"
type,biosphere,biosphere
refobj,93a60a56-a3c8-11da-a746-0800200b9a66,93a60a56-a3c8-11da-a746-0800200b9a66
exchanges_internal_id,8,6
exchanges_name,"methane (Emissions to air, unspecified)","carbon dioxide (Emissions to air, unspecified)"
exchanges_direction,Output,Output
amount,0.0,0.022122


First lets try to pass from ilcd 2.0 to ilcd3

In [12]:
ilcd2_ilcd3 = pd.read_excel('ChangeLog_COMPLETE_EF2.0toEF3.0(1).xlsx',sheet_name='Mapped flows',skiprows=2)
ilcd2_ilcd3

,UUID OLD (EF 2.0),name,UUID NEW (EF 3.0),name.1
0,30da22c2-c0b1-467e-a491-3a07d6d72370,HFE-236ca12 (HG-10),d86b9e71-6555-11dd-ad8b-0800200c9a66,HG-10
1,229e91a6-ff71-4eeb-b71b-c66e7fae9215,Argon-40,3f5f02ed-d820-47b1-b319-505ab06b9bc0,Argon-40
2,549c62f1-9693-4da3-814e-d31ca726837a,Argon-40,e8d5b530-d495-4f79-8c34-ca8ec4852bfa,Argon-40
3,041fd22c-6556-11dd-ad8b-0800200c9a66,polonium,cd375ebd-60f8-4260-b1bd-41b2840376cf,polonium
4,73e2c735-8a30-4f18-a846-d7392081a665,Water (evapotranspiration),4b772bba-b5c1-4098-9c1d-3d9d8c74c7fa,Water (evapotranspiration)
...,...,...,...,...
248,c753193c-87e1-4d85-9fa3-61c6662a5302,beta-pinene,041f840a-6556-11dd-ad8b-0800200c9a66,beta-pinene
249,cd46eb6c-8e97-49c3-bb75-19a4483d93b9,beta-pinene,041f840d-6556-11dd-ad8b-0800200c9a66,beta-pinene
250,d1298572-2991-40ff-8c15-8b7141e198e4,beta-pinene,041f8409-6556-11dd-ad8b-0800200c9a66,beta-pinene
251,f089349e-eb83-4343-aa33-96f1e745472c,beta-pinene,fe0acd60-3ddc-11dd-a795-0050c2490048,beta-pinene


In [17]:
ilcd2_ilcd3_dict = ilcd2_ilcd3.set_index('UUID OLD (EF 2.0)')['UUID NEW (EF 3.0)'].to_dict()

In [18]:
unlinked_df.loc[unlinked_df['type']=='biosphere','uuid'].isin(ilcd2_ilcd3_dict)

0    False
3    False
Name: uuid, dtype: bool

bad luck

There are a couple of biosphere flows unlinked. Probably there are some issues in the existing ilcd - ecoinvent mapping. 

we can first verify if the UUID's are in the input file https://github.com/UNEP-Economy-Division/GLAD-ElementaryFlowResources/tree/master/Mapping/Input/Flowlists

In [16]:
mapping_input_df = pd.read_csv('ILCD_EFv3.0.csv')

/tmp/ipykernel_10641/3603128068.py:1: DtypeWarning: Columns (10,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  mapping_input_df = pd.read_csv('ILCD_EFv3.0.csv')


In [22]:
existing_uuids = mapping_input_df['Flow UUID'].to_list()

In [24]:
unlinked_df.uuid.isin(existing_uuids)

0    False
1    False
2    False
3    False
Name: uuid, dtype: bool

In [13]:
path_to_mapping_excel_ilcd_ei37 = (Path.cwd()/'bw2io'/ 'data'/'ILCD-EFv3.0-ecoinventEFv3.7.xlsx')

In [14]:
mapping_ilcd_ei3_df = pd.read_excel(path_to_mapping_excel_ilcd_ei37)

In [38]:
mapping_ilcd_ei3_df.sample(3).T.dropna(how='all')

,50113,1821,92923
SourceListName,EFv.3.0,EFv.3.0,EFv.3.0
SourceFlowName,hfe-245fa2,radium-226,5-ethyl-m-cresol
SourceFlowUUID,fe0acd60-3ddc-11dd-a386-0050c2490048,fe0acd60-3ddc-11dd-aba6-0050c2490048,063fd00f-9e5b-44ee-a4a6-97763ae5420f
SourceFlowContext,emission/air/non urban or from high stack,emission/air/unspecified,emission/air/lower stratosphere and upper trop...
SourceUnit,kg,kbq,kg
MapType,NO_MAPPING,FLOWNAME_MANUAL,NO_MAPPING
TargetListName,NaN,Ecoinvent_3.7,NaN
TargetFlowName,NaN,radium-226,NaN
TargetFlowUUID,NaN,b912f4b4-34ac-4ec1-aba1-01b9c83683e5,NaN
TargetFlowContext,NaN,air/unspecified,NaN


In [34]:
mapping_ilcd_ei3_df[(mapping_ilcd_ei3_df.SourceFlowName=='methane')]

,SourceListName,SourceFlowName,SourceFlowUUID,SourceFlowContext,SourceUnit,MatchCondition,ConversionFactor,MapType,TargetListName,TargetFlowName,...,Flow Condition (S),Flow Condition (T),Flow Conf (S),Flow Conf (T),Context Condition (S),Context Condition (T),Context Conf (S),Context Conf (T),Conversion Conf (S),Conversion Conf (T)
15180,EFv.3.0,methane,d86caff4-6555-11dd-ad8b-0800200c9a66,emission/water/sea water,kg,NaN,NaN,FLOWNAME_MANUAL_PROXY,Ecoinvent_3.7,"hydrocarbons, aliphatic, alkanes, unspecified",...,<,<,B,B,'=,'=,A,A,NaN,NaN
15181,EFv.3.0,methane,d86cafee-6555-11dd-ad8b-0800200c9a66,emission/water/fresh water,kg,NaN,NaN,FLOWNAME_MANUAL_PROXY,Ecoinvent_3.7,"hydrocarbons, aliphatic, alkanes, unspecified",...,<,<,B,B,~,~,A,A,NaN,NaN
15182,EFv.3.0,methane,d86caffd-6555-11dd-ad8b-0800200c9a66,emission/water/unspecified long term,kg,NaN,NaN,FLOWNAME_MANUAL_PROXY,Ecoinvent_3.7,"hydrocarbons, aliphatic, alkanes, unspecified",...,<,<,B,B,>,~,A,A,NaN,NaN
15183,EFv.3.0,methane,d86caffa-6555-11dd-ad8b-0800200c9a66,emission/water/unspecified,kg,NaN,NaN,FLOWNAME_MANUAL_PROXY,Ecoinvent_3.7,"hydrocarbons, aliphatic, alkanes, unspecified",...,<,<,B,B,'=,'=,A,A,NaN,NaN
27853,EFv.3.0,methane,d86caff1-6555-11dd-ad8b-0800200c9a66,emission/soil/non agricultural,kg,NaN,NaN,NO_MAPPING,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53961,EFv.3.0,methane,d86cafeb-6555-11dd-ad8b-0800200c9a66,emission/soil/agricultural,kg,NaN,NaN,NO_MAPPING,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80249,EFv.3.0,methane,d86caff7-6555-11dd-ad8b-0800200c9a66,emission/soil/unspecified,kg,NaN,NaN,NO_MAPPING,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
mapping_ilcd_ei3_df[mapping_ilcd_ei3_df.SourceFlowName.str.lower().str.contains('\Amethane')
& (mapping_ilcd_ei3_df.SourceFlowContext=='emission/air/unspecified')]

,SourceListName,SourceFlowName,SourceFlowUUID,SourceFlowContext,SourceUnit,MatchCondition,ConversionFactor,MapType,TargetListName,TargetFlowName,...,Flow Condition (S),Flow Condition (T),Flow Conf (S),Flow Conf (T),Context Condition (S),Context Condition (T),Context Conf (S),Context Conf (T),Conversion Conf (S),Conversion Conf (T)
3100,EFv.3.0,methane (biogenic),fe0acd60-3ddc-11dd-a8e8-0050c2490048,emission/air/unspecified,kg,NaN,NaN,FLOWNAME_MANUAL,Ecoinvent_3.7,"methane, non-fossil",...,'=,'=,A,A,'=,'=,A,A,NaN,NaN
3105,EFv.3.0,methane (fossil),08a91e70-3ddc-11dd-9610-0050c2490048,emission/air/unspecified,kg,NaN,NaN,FLOWNAME_MANUAL,Ecoinvent_3.7,"methane, fossil",...,'=,'=,A,A,'=,'=,A,A,NaN,NaN
3111,EFv.3.0,methane (land use change),c714e1e3-8ece-4ab8-99c9-22a903d999ec,emission/air/unspecified,kg,NaN,NaN,FLOWNAME_MANUAL,Ecoinvent_3.7,"methane, from soil or biomass stock",...,~,'=,B,B,'=,'=,A,A,NaN,NaN
5795,EFv.3.0,methanesulfonic acid,0d89b5ff-62c4-4c17-a4dd-281b28297f3e,emission/air/unspecified,kg,NaN,NaN,CAS,Ecoinvent_3.7,methanesulfonic acid,...,NaN,NaN,NaN,NaN,'=,'=,A,A,NaN,NaN
36583,EFv.3.0,methanesulphonyl chloride,986bb5d5-c3b0-4922-a1c5-13ae678aa4d2,emission/air/unspecified,kg,NaN,NaN,NO_MAPPING,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40357,EFv.3.0,methane disulfonic acid disodium salt,18278f83-7deb-403f-bd16-3dd9f32f3844,emission/air/unspecified,kg,NaN,NaN,NO_MAPPING,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
mapping_ilcd_ei3_df[mapping_ilcd_ei3_df.SourceFlowName.str.lower().str.contains('\Acarbon dioxide')
& (mapping_ilcd_ei3_df.SourceFlowContext=='emission/air/unspecified')]

,SourceListName,SourceFlowName,SourceFlowUUID,SourceFlowContext,SourceUnit,MatchCondition,ConversionFactor,MapType,TargetListName,TargetFlowName,...,Flow Condition (S),Flow Condition (T),Flow Conf (S),Flow Conf (T),Context Condition (S),Context Condition (T),Context Conf (S),Context Conf (T),Conversion Conf (S),Conversion Conf (T)
2521,EFv.3.0,carbon dioxide (biogenic),08a91e70-3ddc-11dd-9c15-0050c2490048,emission/air/unspecified,kg,NaN,NaN,FLOWNAME_MANUAL,Ecoinvent_3.7,"carbon dioxide, non-fossil",...,'=,'=,A,A,'=,'=,A,A,NaN,NaN
2535,EFv.3.0,carbon dioxide (fossil),08a91e70-3ddc-11dd-923d-0050c2490048,emission/air/unspecified,kg,NaN,NaN,FLOWNAME_MANUAL,Ecoinvent_3.7,"carbon dioxide, fossil",...,'=,'=,A,A,'=,'=,A,A,NaN,NaN
2540,EFv.3.0,carbon dioxide (land use change),adcb79f3-89cf-45fb-b0b2-65558cb2af26,emission/air/unspecified,kg,NaN,NaN,FLOWNAME_MANUAL,Ecoinvent_3.7,"carbon dioxide, from soil or biomass stock",...,~,~,B,B,'=,'=,A,A,NaN,NaN


In [28]:
mapping_ilcd_ei3_df[mapping_ilcd_ei3_df.SourceFlowUUID.isin(unlinked_df.uuid.to_list())]

,SourceListName,SourceFlowName,SourceFlowUUID,SourceFlowContext,SourceUnit,MatchCondition,ConversionFactor,MapType,TargetListName,TargetFlowName,...,Flow Condition (S),Flow Condition (T),Flow Conf (S),Flow Conf (T),Context Condition (S),Context Condition (T),Context Conf (S),Context Conf (T),Conversion Conf (S),Conversion Conf (T)


In [13]:
unlinked_df.loc[unlinked_df['type']=='technosphere',:].T

,1,2
uuid,55a4c166-2eb6-43a3-9a13-2e4f2c4fee60,e9fd08fb-8067-4d90-91e3-4c92fc1803f7
category_0,Valuable substances,Others
category_1,Energy carrier,None
category_2,Fuels,None
type,technosphere,technosphere
refobj,93a60a56-a3c8-11da-a746-0800200b9a66,93a60a56-a3c8-11da-a746-0800200b9a66
exchanges_internal_id,1,0
exchanges_name,Diesel,Cargo
exchanges_direction,Input,Input
amount,0.007325,1.0


In [13]:
so.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [14]:
so.write_database()

Title: Writing activities to SQLite3 database:
  Started: 02/06/2023 00:16:07
  Finished: 02/06/2023 00:16:07
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.32
Created database: example_glad


Brightway2 SQLiteBackend: example_glad

In [15]:
glad_example = bw2data.Database('example_glad').random()

parameters are parsed although not linked to exchanges

In [16]:
pd.DataFrame(glad_example['parameters']) # not yet linked to exchanges

,name,comment,formula,mean,minimum,maximum,std
0,distance,"[km] distance start - end, default = 100 km",share_mw+share_ru+share_ur,100,0.8,1.5,0
1,payload,"[t] default = 1,2 t",0.000179963579284959/(payload*1000*utilisation),1.2,0,2000,0
2,ppm_sulfur,"[ppm] sulphur content in diesel, default Europ...",0.000158765105879866/(payload*1000*utilisation),10,0,1,0
3,share_check,Check - value must be 1,((share_mw*spec_benzene_mw)+(share_ru*spec_ben...,1,0,1,0
4,share_CO2_bio,[-] share of biogenic C in fuel,0.000214096377021633/(payload*1000*utilisation),0.05,0,1,0
5,share_mw,"[-] driving share motorway , default = 0,29",0.00025863031623885/(payload*1000*utilisation),0.29,0,1,0
6,share_ru,"[-] driving share rural , default = 0,26",0.000228165430598892/(payload*1000*utilisation),0.26,0,1,0
7,share_ur,"[-] driving share urban , default = 0,45",((share_mw*spec_CH4_mw)+(share_ru*spec_CH4_ru)...,0.45,None,None,0
8,spec_benzene_mw,[gbenzene/(kg*km)] benzene emission motorway,0.000307683425489813/(payload*1000*utilisation),1.76434881651921E-7,None,None,0
9,spec_benzene_ru,[gbenzene/(kg*km)] benzene emission rural,0.0133990757167339/(payload*1000*utilisation),1.55652064588104E-7,None,None,0


In [17]:
# contacts are available too
pd.DataFrame(glad_example['contacts'])

,email,website,short_name
0,None,None,GaBi user community
1,None,None,thinkstep
2,info@thinkstep.com,http://www.thinkstep.com,GaBi bug forum
3,info@thinkstep.com,http://www.thinkstep.com,GaBi user forum
4,None,None,No official approval by producer or operator
5,gabi@ibp.fraunhofer.de,http://www.ibp.fraunhofer.de/en/Expertise/Life...,IBP-GaBi
6,unep.tie@unep.org,www.lcinitiative.unep.fr,UNEP SETAC Life Cycle Initiative
7,IES-contact@jrc.ec.europa.eu,http://ies.jrc.ec.europa.eu,"European Commission, JRC-IES"
8,info@thinkstep.com,http://www.thinkstep.com,thinkstep
9,gabi@lbp-gabi.de,http://www.LBPGABI.uni-stuttgart.de,IABP-GaBi


In [18]:
glad_example['general_comment']

'- average emission values Euro 3\n- source emissions/driving share: HBEFA 3.1, status January 2010\n- input parameter: distance [km], payload [t], utilisation [-], sulphur content diesel [ppm], driving share motorway/rural/urban\n- average sulphur content: EU = 10 ppm'

In [22]:
glad_example.get('licensetype')

'Other'

In [23]:
glad_example.get('dataset_format')

'ILCD format 1.1'

that may explain the problems with linking.

In [24]:
lca = bw2calc.LCA({glad_example:1},gwp100)
lca.lci()
lca.lcia()
lca.score

-0.0008142366941782204

In [25]:
for e in glad_example.exchanges():
    print(e)

Exchange: 1.86089936452096e-08 kilogram 'Benzene' (kilogram, None, ('air',)) to 'Light duty vehicle' (kilogram, GLO, None)>
Exchange: 4.08539839600231e-06 kilogram 'Carbon monoxide, fossil' (kilogram, None, ('air',)) to 'Light duty vehicle' (kilogram, GLO, None)>
Exchange: 5.93778826661554e-06 kilogram 'Particulates, < 2.5 um' (kilogram, None, ('air',)) to 'Light duty vehicle' (kilogram, GLO, None)>
Exchange: 4.77321533595815e-07 kilogram 'Dinitrogen monoxide' (kilogram, None, ('air',)) to 'Light duty vehicle' (kilogram, GLO, None)>
Exchange: 4.77250386102527e-05 kilogram 'Nitrogen oxides' (kilogram, None, ('air',)) to 'Light duty vehicle' (kilogram, GLO, None)>
Exchange: 8.86322159977519e-05 kilogram 'Nitric oxide' (kilogram, None, ('air',)) to 'Light duty vehicle' (kilogram, GLO, None)>
Exchange: 0.00116432140873928 kilogram 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)) to 'Light duty vehicle' (kilogram, GLO, None)>
Exchange: 9.80392203428873e-08 kilogram 'Ammonia' (kilogra

## lcacommons data

from https://www.lcacommons.gov/

In [26]:

path_to_example = Path('bw2io/data/examples/ilcd_examples/f29030e5-f681-31ae-9be8-d47f905cb0cb_3281fd83-15b3-4183-997d-dda9ea50dacd.zip')
lcacommons = ILCDImporter(dirpath= path_to_example,dbname='example_ilcd_lcacommons')


In [27]:
lcacommons.apply_strategies()

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applying strategy: set_activity_parameters
Applied 8 strategies in 0.03 seconds


In [28]:

lcacommons.match_database('biosphere3',fields=['database','code'])
lcacommons.match_database(fields=['database','code',])
lcacommons.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
2128 exchanges
2127 unlinked exchanges
  Type biosphere: 1135 unique unlinked exchanges
  Type technosphere: 103 unique unlinked exchanges


(1, 2128, 2127)

In [8]:
lcacommons.data[0].get('name')

'coal extraction and processing - Northern Appalachia, WC, Underground'

In [9]:
lcacommons_df = pd.DataFrame(lcacommons.data[0]['exchanges'])

Instead of using the ilcd to ecoinvent 3.7 mapping, lets use the `FEDEFL to ecoinvent`

In [29]:
path_to_alt_mapping = (Path.cwd() /'bw2io'/'data'/'FEDEFLv1.0.3-ecoinventEFv3.7.xlsx')
path_to_alt_mapping.exists()

True

In [30]:
lcacommons.data = bw2io.strategies.ilcd.alternative_map_to_biosphere3(lcacommons.data,
path_to_alt_mapping)

In [31]:
lcacommons.match_database('biosphere3',fields=['database','code'])

Applying strategy: link_iterable_by_fields


In [32]:
lcacommons.statistics()

1 datasets
2128 exchanges
1051 unlinked exchanges
  Type biosphere: 752 unique unlinked exchanges
  Type technosphere: 103 unique unlinked exchanges


(1, 2128, 1051)

In [14]:
for i,u in enumerate(lcacommons.unlinked):
    print(f"{u['type']} --- {u['name']} --- {u['uuid']}")

    if i > 10: break

technosphere --- 2,4-DINITROTOLUENE --- d3540cb8-a801-3913-9040-1619ab5b3b46
biosphere --- Methyl parathion --- 0cf75565-4164-3e75-b847-4974ebe979fc
biosphere --- Bicarbonate --- 1208f3d7-8e09-3e47-9fa8-cbef9b54253d
biosphere --- 2,4-D isopropyl ester --- 2fda9070-a8d8-3316-993e-42179ddaf687
biosphere --- Nitroglycerin --- 8ee2febb-ace4-38cc-b478-a62b5cf3ed1d
biosphere --- Benfluralin --- 076a10b9-d926-33f7-b146-10a1e778d9e7
biosphere --- Chromic(VI) acid --- 4717a8b5-2466-3dd1-b856-f639e8442e75
biosphere --- Pyrethrin I --- b61e086a-463f-3b36-b45d-649657392231
biosphere --- Cupric sulfate --- e9be3546-c54f-3ebc-96a2-a8897de5755f
biosphere --- Dimethipin --- 13a4f088-d48e-3c23-bd72-7b74d23a190e
biosphere --- (E)-Crotonic acid --- 9d718e3c-b210-3b88-8d2e-27982bcc69e8
technosphere --- CORROSIVE WASTE --- d5d165f8-aad3-3b82-a849-1d158c5d031d


In [33]:
lcacommons.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.02 seconds


In [34]:
postdrop_df = pd.DataFrame(lcacommons.data[0]['exchanges'])

In [35]:
Counter(postdrop_df['type'])

Counter({'biosphere': 1076, 'production': 1})

In [36]:
postdrop_df.sample(3).T

,589,898,234
uuid,3e9e5cb3-0f07-3618-ac5c-5d89626970b1,f04bdc59-0f29-3c7f-956c-53b0003b37e7,7a29ec10-2ef3-38d3-a7a7-949b39bcd19b
category_0,Elementary flows,Elementary flows,Elementary flows
category_1,emission,emission,emission
category_2,ground,water,water
type,biosphere,biosphere,biosphere
refobj,93a60a56-a3c8-11da-a746-0800200b9a66,93a60a56-a3c8-11da-a746-0800200b9a66,93a60a56-a3c8-11da-a746-0800200b9a66
exchanges_internal_id,885,801,1110
exchanges_name,Chromium compounds,Isoprene,m-Cresol
exchanges_direction,Output,Output,Output
amount,0.000017,0.0,0.0


In [37]:
lcacommons.write_database()

Title: Writing activities to SQLite3 database:
  Started: 02/06/2023 00:20:14
  Finished: 02/06/2023 00:20:15
  Total time elapsed: 00:00:00
  CPU %: 73.30
  Memory %: 4.77
Created database: example_ilcd_lcacommons


Brightway2 SQLiteBackend: example_ilcd_lcacommons

In [38]:
commons_example = bw2data.Database('example_ilcd_lcacommons').random()

In [39]:
surprise = bw2data.Database('example_ilcd_lcacommons').random()
surprise

'coal extraction and processing - Northern Appalachia, WC, Underground' (kilogram, GLO, None)

In [40]:
lca = bw2calc.LCA({surprise:1},gwp100)
lca.lci()
lca.lcia()
lca.score

364.82272957334135

In [41]:
surprise.as_dict().keys()

dict_keys(['treatment_standards_routes', 'mix_and_location_types', 'functional_unit_flow_properties', 'uuid', 'general_comment', 'reference_year', 'data_set_valid_until', 'time_representativeness_description', 'location', 'reference_to_reference_flow', 'parameter_name', 'parameter_comment', 'parameter_mean_value', 'parameter_minimum_value', 'parameter_maximum_value', 'parameter_std95', 'parameter_formula', 'intended_application', 'dataset_format', 'licensetype', 'contacts', 'database', 'type', 'code', 'name', 'unit', 'exchanges_name', 'reference product', 'production amount'])

In [42]:
surprise.get('general_comment')

'The cradle-to-gate inventory for production of coal aggregated to basin, mine type, and coal type groups. For coal extraction there are two major processes that form the basis of the coal life cycle model - underground and surface coal mining. These are connected to auxiliary processes that provide inventories from things like coal mine methane emissions, water use, water emissions, etc. All processes use parameters that allow some differentiation based on region or coal type. Details on the coal modeling can be found in the NETL Coal Baseline report to be published in the near future: netl.doe.gov/LCA This process was created with ElectricityLCI (https://github.com/USEPA/ElectricityLCI) version 1.0.1 using the ELCI_1 configuration.'

In [43]:
surprise.get('contacts')

[{'email': None, 'website': None, 'short_name': None},
 {'email': None, 'website': None, 'short_name': None}]

In [44]:
surprise.get('parameter_name')

In [45]:
surprise.get('licensetype')

In [46]:
surprise.get('dataset_format')

'ILCD format'

In [47]:
surprise.get('intended_application')

'The intended application for these inventories is to provide high-resolution electricity data for the establishment of robust background data used in LCIs of US systems. These data were developed as regionally specific, average electricity generation/consumption LCI for use in the application of LCAs where accurate electricity inventories are needed to assess the system of interest and/or to evaluate the sensitivity to changes in supplied electricity. A full inventory of environmental flows are included enabling use of a full range of LCIA impact categories.'

the sign is a bit suspicious

## GLAD example

in the case of datasets from lca commons they seem.. wrong ?

In [23]:
lca.top_emissions()

ImportError: `bw2analyzer` is not installed

## several ILCD files from GLAD

In [19]:
path_to_examples = Path('bw2io/data/examples/ilcd_examples')

In [20]:
assert path_to_examples.is_dir()

In [21]:
paths_to_examples = list(path_to_examples.glob('*zip'))

In [22]:
examples = []
for p in paths_to_examples:

    so = ILCDImporter(dirpath= p,dbname='example_glad_many')
    so.apply_strategies()
    so.match_database('biosphere3',fields=['database','code'])
    so.match_database(fields=['database','code'])
    so.statistics()

    examples.append(so)

Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applying strategy: set_activity_parameters
Applied 8 strategies in 0.02 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1 datasets
199 exchanges
198 unlinked exchanges
  Type biosphere: 133 unique unlinked exchanges
  Type technosphere: 15 unique unlinked exchanges
Applying strategy: rename_activity_keys
Applying strategy: get_activity_unit
Applying strategy: set_production_exchange
Applying strategy: convert_to_default_units
Applying strategy: assign_only_product_as_production
Applying strategy: map_to_biosphere3
Applying strategy: set_default_location
Applying strategy: set_activity_parameters
Applied 8 strategies in 0.02 seconds
Applying strategy: l

In [32]:
for e in examples:
    ilcd2_format = pd.DataFrame(e.unlinked).uuid.isin(ilcd2_ilcd3).sum()
    print(ilcd2_format)

0
0
0
0
0
0
0
0
0
0
0


In [40]:
bw2io.data.get_ilcd_biosphere_migration_data()

{'36637b3e-3131-460d-9ae6-dc2673bed9c1': '24752b90-cc53-4198-a442-14196853148d',
 'ff5f014d-eee7-40cd-879a-15c1b425cb4b': 'e385da6d-8c5a-42ff-a8fe-d18943079774',
 'f43d9114-9ff5-4086-8d0d-df64b78d0af8': 'd069e5a1-c3c0-4513-9fa5-ff3ebe687ea7',
 '12fa3910-3941-40ed-9d19-b86bfd2d92c0': 'e69dfe04-4b45-4e87-a6ce-08b1af61ca22',
 'a51f852d-4417-4055-af65-d1d330b2d484': '24752b90-cc53-4198-a442-14196853148d',
 'ba96ca66-9b78-40cf-a3e1-1039ebfa83e1': '082903e4-45d8-4078-94cb-736b15279277',
 '4f2c2a1f-5260-4c0c-8f3b-4bc9702adf50': 'd1d77d54-e19f-4c3d-8fdc-389f78e72515',
 '1e868b11-3d25-4ffc-aa84-37ce945efb34': 'cce10142-69c4-4a72-a010-ac6b6abad1d3',
 '2b08ca86-10ef-4f3a-aff2-7179d02f483c': 'a7f661f4-da39-4571-a48b-a8d61314ce1b',
 'b05567ff-ac7f-4bcc-9790-37b5c2532794': '237a5f15-8119-472a-8988-88b7ecb42405',
 'cd7e4e40-04aa-43e8-b502-a441d4295f4c': '737c3221-1456-4c6b-8605-6ac00dc88b2e',
 '146a2005-7de4-4fa7-9848-7c9f9d0da2d9': 'f77c5e36-ee47-4437-b757-03139bb1d6d6',
 'fe0acd60-3ddc-11dd-a4a3-00

In [43]:
md = bw2io.data.get_exiobase_biosphere_migration_data()

In [44]:
type(md)

dict

In [45]:
md.keys()

dict_keys(['fields', 'data'])

In [50]:
md['fields']

['name']

In [47]:
md['data'][0]

[['CO2 - combustion'],
 {'name': 'Carbon dioxide, fossil', 'categories': ['air']}]

key - to fields to fill

In [49]:
bw2io.data.get_biosphere_2_3_category_migration_data()['data'][0]

((('resource', 'in ground'), 'biosphere'),
 {'categories': ('natural resource', 'in ground')})

In [52]:
bw2io.data.get_simapro_ecoinvent_3_migration_data()

TypeError: get_simapro_ecoinvent_3_migration_data() missing 1 required positional argument: 'version'

In [34]:
bw2io.migrations['simapro-ecoinvent-3.5']

{'description': 'Change SimaPro names from ecoinvent 3.5 to ecoinvent names'}

In [28]:
for e in examples:
    e.drop_unlinked(True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.05 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.17 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds
Applying strategy: drop_unlinked
Applied 1 strategies in 0.00 seconds


In [57]:
all_data = []
existing_uuids = []
for e in examples:
    for dataset in e.data:
        
        if dataset['uuid'] not in existing_uuids:
    
            all_data.append(dataset)
            existing_uuids.append(dataset['uuid'])

In [58]:
len(all_data)

550

In [59]:
so.write_database(data=all_data)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 02/04/2023 23:54:39
  Finished: 02/04/2023 23:54:40
  Total time elapsed: 00:00:00
  CPU %: 83.10
  Memory %: 16.88
Created database: example_glad_many


Brightway2 SQLiteBackend: example_glad_many

In [60]:
ilcd_db = bw2data.Database('example_glad_many')

In [66]:
for act in ilcd_db:

    lca = bw2calc.LCA({act:1},gwp100)
    lca.lci()
    lca.lcia()

    if lca.score != 0:
        print(f"one {act['unit']} of {act['name']}, has an CO2 footprint of {lca.score:.2f} kg CO2eq")

one kilogram of Polycarbonate, has an CO2 footprint of 3.42 kg CO2eq
one kilogram of Short Chain Polyether Polyols (rigid), has an CO2 footprint of 3.19 kg CO2eq
one kilogram of Cobalt, refined (metal), has an CO2 footprint of 1.56 kg CO2eq
one kilogram of Aluminium ingot (AlSi7Mg) sec., has an CO2 footprint of 0.03 kg CO2eq
one kilogram of Light duty vehicle, has an CO2 footprint of -0.00 kg CO2eq
